# Initial Data Cleaning and Split
In this notebook, I have done a basic exploration and cleaning on the raw dataset. This was done to ensure that the current data is fit for further exploration and analysis. I have tried my best to avoid data leakage at this stage. As a result of this, I was only able to do trivial exploration and cleaning procedures. This dataset will be explored and cleaned(if necessary) rigorously in the later stages. Also, I have split the raw data into train and test set.

In [1]:
# Importing required libraries and modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Data Loading

In [2]:
raw = pd.read_csv('../data/raw/internet_service_churn.csv')

In [3]:
print('Shape=>',raw.shape)
raw.head()

Shape=> (72274, 11)


,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


This dataset contains 11 columns and 72274 rows. The description of each of these columns are as follows:
1. **id:** Unique subscriber ID
2. **is_tv_subscriber:** Does customer has a TV subscription ?
3. **is_movie_package_subscriber:** Is he/she has a cinema movie package subscriber?
4. **subscription_age:** How many year has the customer used our service?
5. **bill_avg:** Last 3 months average bill
6. **reamining_contract:** How many year remaining for customer contract? *(If null, customer don't have a contract. The customer who has a contract time has to use their service until contract ends. If they cancel their service before contract time ends then they have to pay a penalty fare)*
7. **service_failure_count:** Count of customer calls to call center for service failure in the last 3 months
8. **download_avg:** Last 3 months average download (GB)
9. **upload_avg:** Last 3 months average upload (GB)
10. **download_over_limit:** Count of download limit reached in the past 9 months. *(Most of the customers have a download limit. If they reach this limit they have to pay for that.)*
11. **churn:** This is the target column. If the customer has cancelled the service then it is 1 else 0.

## Basic Data Exploration

In [4]:
# Getting an overview of the dataset
raw.info(show_counts=True,verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72274 entries, 0 to 72273
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           72274 non-null  int64  
 1   is_tv_subscriber             72274 non-null  int64  
 2   is_movie_package_subscriber  72274 non-null  int64  
 3   subscription_age             72274 non-null  float64
 4   bill_avg                     72274 non-null  int64  
 5   reamining_contract           50702 non-null  float64
 6   service_failure_count        72274 non-null  int64  
 7   download_avg                 71893 non-null  float64
 8   upload_avg                   71893 non-null  float64
 9   download_over_limit          72274 non-null  int64  
 10  churn                        72274 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 6.1 MB


There is a typo in a column name so let's fix that first.

In [5]:
raw.rename(columns={'reamining_contract' : 'remaining_contract'}, inplace = True)

Let's now check for duplicates in the dataset.

In [6]:
# Number of duplicate rows in dataframe
print(raw.duplicated().sum())

0


It's good to see that we have no duplicates in our dataset. Let's now look out for columns with high percentage of missing values.

In [7]:
# Pecentage of missing values per column
missing_percentage=raw.isna().mean().round(3)*100
missing_percentage

id                              0.0
is_tv_subscriber                0.0
is_movie_package_subscriber     0.0
subscription_age                0.0
bill_avg                        0.0
remaining_contract             29.8
service_failure_count           0.0
download_avg                    0.5
upload_avg                      0.5
download_over_limit             0.0
churn                           0.0
dtype: float64

Our dataset except *remaining_contract* column has very few missing values so we don't have to remove any column for now and we can handle them in later stages of the project. Next, let's see the number of unique values in every column. This will help us in finding zero-variance predictors.

In [8]:
# Number of unique values per column
raw.nunique()

id                             72274
is_tv_subscriber                   2
is_movie_package_subscriber        2
subscription_age                1110
bill_avg                         179
remaining_contract               247
service_failure_count             19
download_avg                    2856
upload_avg                       802
download_over_limit                8
churn                              2
dtype: int64

None of the columns contain a single value so now we can move forward. Let's see the distribution of target variable.

In [9]:
raw['churn'].value_counts(normalize=True)*100

1    55.414118
0    44.585882
Name: churn, dtype: float64

Our dataset is slightly imbalanced but it's not a matter of serious concern.

## Splitting Dataset
Next, I will split the raw dataset into train and test sets. Our training and test sets will contain 62274 and 10000 instances, respectively.

In [10]:
# Creating Train and Test Sets
train_df, test_df = train_test_split(raw, stratify=raw['churn'],test_size=10000,random_state=42)

In [11]:
print('Shape of Training set=>', train_df.shape)
train_df.head()

Shape of Training set=> (62274, 11)


,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,remaining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
53349,1250624,1,1,1.77,7,0.19,0,114.1,8.7,0,0
71329,1668176,1,0,0.05,6,0.59,0,12.7,1.3,0,0
49035,1151155,0,0,1.42,18,NaN,0,0.4,0.0,0,1
51987,1219428,1,0,0.73,20,0.00,0,9.3,0.4,0,1
57420,1344910,1,0,0.25,17,0.00,0,6.1,0.5,0,1


In [12]:
print('Shape of Test set=>', test_df.shape)
test_df.head()

Shape of Test set=> (10000, 11)


,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,remaining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
50643,1187227,1,1,2.05,14,0.61,0,30.2,2.3,0,0
20608,481052,1,0,3.19,7,0.00,0,7.0,0.6,0,1
17528,409792,1,1,6.75,0,NaN,0,0.0,0.0,0,1
40765,954134,1,0,3.14,12,0.81,0,189.5,15.4,0,0
16347,380945,1,0,6.45,21,0.19,0,20.2,1.4,0,0


In [13]:
train_df['churn'].value_counts(normalize=True)

1    0.554148
0    0.445852
Name: churn, dtype: float64

In [14]:
test_df['churn'].value_counts(normalize=True)

1    0.5541
0    0.4459
Name: churn, dtype: float64

## Exporting Datasets

In [15]:
train_df.to_csv('../data/interim/train.csv', index=False)
test_df.to_csv('../data/interim/test.csv', index=False)